# Taxi Gridworld Agent with OpenAI Gymnasium (Taxi-v3)

This project trains and evaluates a simple reinforcement-learning-ready agent in the classic Taxi gridworld environment using OpenAI Gymnasium’s Taxi-v3 task. It provides a minimal agent–environment interaction loop, performance monitoring, and a clean starting point for implementing Q-learning, SARSA, or other value-based methods.

## Overview

This project implements a minimal RL agent interacting with the classic **Taxi** environment, originally from OpenAI Gym and now available via **Gymnasium** as `Taxi-v3`. The agent maintains a state–action table `Q[s][a]` (initialized via `defaultdict`) and runs for many episodes to collect rewards, making it a clean starting point for experimenting with **Q-learning, SARSA, and other tabular RL algorithms** in a discrete gridworld.

The environment is a **5×5 grid** with walls, four landmark locations (**R, G, Y, B**), and a single taxi that must:
1. Navigate to the passenger,
2. Pick them up,
3. Navigate to the destination,
4. Drop them off.

The key details (as defined in the Taxi environment and Dietterich’s MAXQ paper) are:

- **Action space (6 discrete actions)**  
  `0` = South, `1` = North, `2` = East, `3` = West, `4` = Pickup, `5` = Dropoff  

- **State space (500 discrete states)**  
  Encodes: taxi row (5) × taxi column (5) × passenger location (5: R, G, Y, B, in-taxi) × destination (4: R, G, Y, B) → **500 states total**.

- **Rewards**  
  - `-1` per time step (including bumps into walls) → encourages shortest routes  
  - `+20` for successful drop-off at correct destination (episode ends)  
  - `-10` for illegal pickup/dropoff actions  

The provided `interact` loop tracks **average reward over a sliding window** and prints the **best average reward**, stopping once the task is considered solved (e.g., average reward ≥ 9.7 over the last 100 episodes).

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym  # modern replacement for gym


class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent with novelty-biased exploration.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> np.array of action values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Path memory: counts how often each action is taken in each state recently.
        # Used to bias exploration toward less-frequently-taken actions.
        self.path_counts = defaultdict(lambda: np.zeros(self.nA))

        # --- Hyperparameters (tuned for Taxi-v3) ---
        self.alpha = 0.1          # initial learning rate
        self.alpha_min = 0.01     # minimum learning rate
        self.alpha_decay = 0.9995 # decay per episode

        self.gamma = 0.99         # discount factor

        # ε-greedy with novelty-biased softmax for exploration
        self.epsilon = 1.0        # initial exploration rate
        self.epsilon_min = 0.05   # minimum exploration rate
        self.epsilon_decay = 0.9995  # decay per episode

        # Softmax exploration parameters when we *do* explore
        self.beta_q = 3.0         # weight for Q-values in softmax
        self.beta_novelty = 1.0   # weight for novelty (path_counts) in softmax
        self.path_decay = 0.9     # geometric decay of path memory per episode

        self.episode = 0          # count completed episodes

    def _softmax_explore(self, state):
        """Sample an action using softmax over (Q - novelty)."""
        q_vals = self.Q[state]
        counts = self.path_counts[state]

        # Higher Q is good, higher counts mean "already tried a lot" → subtract them.
        prefs = self.beta_q * q_vals - self.beta_novelty * counts

        # Numerical stability
        prefs = prefs - np.max(prefs)
        exp_prefs = np.exp(prefs)
        probs = exp_prefs / np.sum(exp_prefs)

        return np.random.choice(self.nA, p=probs)

    def select_action(self, state):
        """Given the state, select an action using ε-greedy with novelty-biased exploration.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        # Exploit (greedy) with probability 1 - ε
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[state])

        # Explore with probability ε: softmax over (Q - novelty)
        return self._softmax_explore(state)

    def _end_of_episode_update(self):
        """Updates done once per episode: decay ε, α, and path memory."""
        self.episode += 1

        # Decay epsilon (exploration rate), but keep it above epsilon_min
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

        # Decay learning rate slightly toward alpha_min
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)

        # Geometric decay for path memory: keeps it focused on *recent* behavior
        for s in self.path_counts:
            self.path_counts[s] *= self.path_decay

    def step(self, state, action, reward, next_state, done):
        """Update the agent's knowledge using the Q-learning update rule.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        # Standard Q-learning target
        best_next_action_value = 0.0 if done else np.max(self.Q[next_state])
        td_target = reward + self.gamma * best_next_action_value
        td_error = td_target - self.Q[state][action]

        # Q-learning update
        self.Q[state][action] += self.alpha * td_error

        # Update path memory (for novelty-based exploration)
        self.path_counts[state][action] += 1.0

        # If episode finished, perform per-episode updates (ε, α, path memory)
        if done:
            self._end_of_episode_update()

In [ ]:
def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.3f} α={agent.alpha:.3f}",
            end=""
        )
        sys.stdout.flush()

        # "Solved" threshold (as in Udacity / Gym examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

        if i_episode == num_episodes:
            print("\n")

    return avg_rewards, best_avg_reward

In [ ]:
# ==== main notebook entry point ====
env = gym.make("Taxi-v3")  # Gymnasium Taxi-v3
agent = Agent(nA=env.action_space.n)

avg_rewards, best_avg_reward = interact(env, agent)
print("\n\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 8.71 || ε=0.050 α=0.010



Final best average reward: 8.71


Experiment 2:

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym  # updated: use gymnasium instead of gym


class Agent:

    def __init__(self, nA=6):
        """Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Hyperparameters for Q-learning
        self.alpha = 0.1  # learning rate
        self.gamma = 0.99  # discount factor

        # Epsilon-greedy parameters
        self.epsilon = 1.0  # initial exploration rate
        self.epsilon_min = 0.01  # minimum exploration rate
        self.epsilon_decay = 0.9999  # decay rate per episode

        self.episode_count = 0

    def select_action(self, state):
        """Given the state, select an action using epsilon-greedy policy.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        # Epsilon-greedy action selection
        if np.random.random() < self.epsilon:
            # Explore: choose random action
            return np.random.choice(self.nA)
        else:
            # Exploit: choose best action based on current Q-values
            return np.argmax(self.Q[state])

    def step(self, state, action, reward, next_state, done):
        """Update the agent's knowledge using Q-learning update rule.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        # Q-learning update rule:
        # Q(s,a) ← Q(s,a) + α[r + γ·max Q(s',a') - Q(s,a)]

        if done:
            # Terminal state: no future rewards
            target = reward
        else:
            # Use max Q-value of next state for update
            target = reward + self.gamma * np.max(self.Q[next_state])

        # Update Q-value
        self.Q[state][action] += self.alpha * (target - self.Q[state][action])

        # Decay epsilon after each step
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || Best average reward {best_avg_reward:.2f}",
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
# create environment and agent, then run training
env = gym.make("Taxi-v3")  # updated: Taxi-v3 with Gymnasium
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 8.37



Improvement of the above:

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym  # updated: use gymnasium instead of gym


class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        # Zero init (this matched your best grid-search config)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.1      # learning rate (fixed)
        self.gamma = 0.99     # discount factor

        # Epsilon-greedy exploration hyperparameters
        self.epsilon = 1.0        # initial exploration rate
        self.epsilon_min = 0.001  # minimum exploration rate (fast-decay config)
        self.epsilon_decay = 0.995  # decay per EPISODE (not per step!)

        self.episode_count = 0    # counts completed episodes

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        With probability epsilon: choose a random action (explore).
        With probability 1 - epsilon: choose greedy action wrt Q (exploit).
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _end_of_episode_update(self):
        """Decay epsilon once per episode."""
        self.episode_count += 1
        # Per-episode epsilon decay schedule (fast decay)
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update using the most recent transition.

        Q(s,a) ← Q(s,a) + α [ r + γ max_a' Q(s',a') - Q(s,a) ]
        """
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            # No future value if this is a terminal transition
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # If the episode has finished, do per-episode housekeeping (epsilon decay)
        if done:
            self._end_of_episode_update()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.3f}",
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")  # updated: Taxi-v3 with Gymnasium
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)


Episode 20000/20000 || Best average reward 8.62 || ε=0.001


Final best average reward: 8.62


## Novelty-Softmax Q-Learning

Standard Q-learning + ε-greedy, but when exploring, we sample actions from a softmax that prefers (1) higher-Q actions and (2) less-used actions in that state, tracked via a decaying path-memory table. This is inspired by Andy Harless’ leaderboard solution and is designed to squeeze out those last +0.5–1.0 average reward points.

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym  # updated: use gymnasium instead of gym


class Agent:
    def __init__(self, nA=6):
        """Novelty-Softmax Q-learning agent for Taxi-v3.

        - Base algorithm: tabular Q-learning
        - Exploration: epsilon-greedy at top level
          * Greedy: argmax_a Q(s,a)
          * Exploratory: softmax over (beta_q * Q(s,a) - beta_n * path_counts(s,a))
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Path memory: how often each action was taken in each state (recently).
        # This is used as a "novelty penalty" in exploration.
        self.path_counts = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.1         # initial learning rate
        self.alpha_min = 0.05    # don't go below this (small decay for stability)
        self.alpha_decay = 0.999 # per-episode decay (very gentle)
        self.gamma = 0.99        # discount factor

        # Epsilon-greedy exploration hyperparameters (per EPISODE)
        self.epsilon = 1.0         # initial exploration rate
        self.epsilon_min = 0.001   # minimum exploration rate
        self.epsilon_decay = 0.995 # fast decay (your best config)

        # Softmax exploration hyperparameters
        self.beta_q = 3.0          # weight for Q-values in softmax
        self.beta_novelty = 1.0    # weight for novelty penalty (path_counts)
        self.path_decay = 0.9      # geometric decay of path_counts per episode

        self.episode_count = 0

    # ---------- Action selection helpers ----------

    def _greedy_action(self, state):
        """Return greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def _softmax_explore_action(self, state):
        """Sample an exploratory action via softmax over (Q - novelty)."""
        q_vals = self.Q[state]
        counts = self.path_counts[state]

        # Preferences: high Q is good; high count (over-used) is bad.
        prefs = self.beta_q * q_vals - self.beta_novelty * counts

        # Numerical stability
        prefs = prefs - np.max(prefs)
        exp_prefs = np.exp(prefs)
        probs = exp_prefs / np.sum(exp_prefs)

        return np.random.choice(self.nA, p=probs)

    def select_action(self, state):
        """Select an action using epsilon-greedy with novelty-softmax exploration.

        With probability epsilon: explore (softmax over Q - novelty).
        With probability 1 - epsilon: exploit (greedy wrt Q).
        """
        if np.random.random() < self.epsilon:
            return self._softmax_explore_action(state)
        else:
            return self._greedy_action(state)

    # ---------- Episode-level housekeeping ----------

    def _end_of_episode_update(self):
        """Decay epsilon, alpha, and path memory once per episode."""
        self.episode_count += 1

        # Per-episode epsilon decay
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

        # Gentle alpha decay (keeps early learning fast, late learning stable)
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)

        # Geometric decay of path_counts to emphasize RECENT behavior
        for s in self.path_counts:
            self.path_counts[s] *= self.path_decay

    # ---------- Learning update ----------

    def step(self, state, action, reward, next_state, done):
        """Q-learning update using the most recent transition.

        Q(s,a) ← Q(s,a) + α [ r + γ max_a' Q(s',a') - Q(s,a) ]
        """
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error and update
        td_error = target - q_sa
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Update path memory (for novelty-aware exploration)
        self.path_counts[state][action] += 1.0

        # If episode finished, do per-episode housekeeping
        if done:
            self._end_of_episode_update()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.3f} α={agent.alpha:.3f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 9.01 || ε=0.001 α=0.050


Final best average reward: 9.01


## Gemini: Optimized Epsilon-Decay Q-Learning

This revised agent uses a slower, step-based decay for the $\epsilon$ (exploration) parameter, allowing for a more sustained exploration period. It also adjusts the epsilon_decay and alpha hyperparameters based on common successful configurations for the Taxi-v3 environment

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.08     # Learning rate: Slightly reduced from 0.1
        self.gamma = 0.99     # Discount factor (kept high)

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration: 0.99995 is common
        self.epsilon_decay = 0.99995

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}", # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3", render_mode=None)
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 9.14 || ε=0.00100


Final best average reward: 9.14


## Variation of the above with more episodes

In [ ]:
# Trying the same as above but with more episodes and a few more changes
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Q-Learning with Andy-like Hyperparams & Epsilon Schedule ===

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3 with tuned hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters (aligned with Andy's setup)
        self.alpha = 0.7      # learning rate (aggressive, as in Andy's code)
        self.gamma = 0.5      # discount factor (shorter-sighted, as in Andy)

        # Epsilon schedule parameters (per EPISODE, like Andy)
        self.a = -0.005
        self.b = 5e-5
        self.epsilon_min = 0.0

        # Track episodes to drive epsilon schedule
        self.episode_idx = 0

        # Initial epsilon (episode 0)
        self.epsilon = max(self.epsilon_min, math.exp(self.a - self.b * self.episode_idx))

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy."""
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon_episode(self):
        """Update epsilon once per episode using Andy-style schedule."""
        self.episode_idx += 1
        self.epsilon = max(self.epsilon_min,
                           math.exp(self.a - self.b * self.episode_idx))

    def step(self, state, action, reward, next_state, done):
        """Q-learning update; epsilon decay happens per EPISODE."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # If the episode ended, update epsilon for the *next* episode
        if done:
            self._update_epsilon_episode()


def interact(env, agent, num_episodes=150000, window=100):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")  # Gymnasium env id
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 150000/150000 || Best average reward 8.93 || ε=0.00055


Final best average reward: 8.93


In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.08     # Learning rate: Slightly reduced from 0.1
        self.gamma = 0.99     # Discount factor (kept high)

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration: 0.99995 is common
        self.epsilon_decay = 0.99995

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=150000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",  # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")  # <- fixed env id
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 150000/150000 || Best average reward 9.05 || ε=0.00100


Final best average reward: 9.05


In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym
from gymnasium.envs.toy_text.taxi import TaxiEnv

# Register an alias "Taxi-v3a" that uses the same TaxiEnv
gym.register(
    id="Taxi-v3a",
    entry_point="gymnasium.envs.toy_text.taxi:TaxiEnv",
)

env = gym.make("Taxi-v3a")


# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.08     # Learning rate: Slightly reduced from 0.1
        self.gamma = 0.99     # Discount factor (kept high)

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration: 0.99995 is common
        self.epsilon_decay = 0.99995

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=150000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}", # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3a")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 150000/150000 || Best average reward 9.19 || ε=0.00100


Final best average reward: 9.19


## Softmax novelty q-learning

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


# === Softmax Novelty Q-Learning Agent =======================================

class Agent:
    def __init__(self, nA=6):
        """Q-learning agent for Taxi-v3 with novelty-aware softmax exploration.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: state -> action-values
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Path memory: counts how often we've taken each action in each state
        # (recently, because we decay it each episode)
        self.path = defaultdict(lambda: np.zeros(self.nA))

        # --- Core Q-learning hyperparameters (kept conservative) -------------
        self.alpha = 0.08       # learning rate (slightly smaller than 0.1)
        self.gamma = 0.99       # discount factor

        # --- Epsilon-greedy exploration (step-based decay) -------------------
        self.epsilon = 1.0          # start fully exploring
        self.epsilon_min = 0.001    # floor
        self.epsilon_decay = 0.99995  # decay per step (slow-ish, your best so far)

        # --- Softmax exploration over (Q - novelty) --------------------------
        # Higher beta_Q = more strongly follow Q-values when exploring
        # Higher beta_P = stronger penalty for over-used actions (path memory)
        self.beta_Q = 1.0
        self.beta_P = 1.0

        # Path memory decay per episode (0<path_decay<1)
        #  -> 0.9 = keep strong memory of current episode, weak of very old ones
        self.path_decay = 0.9

    # -------------------------------------------------------------------------
    # Utility functions
    # -------------------------------------------------------------------------
    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def _softmax_explore_action(self, state):
        """Sample an action using softmax over (beta_Q * Q - beta_P * path)."""
        q_vals = self.Q[state]
        p_vals = self.path[state]

        # Preferences combine "greedy" (Q) and "novelty" (negative path count)
        prefs = self.beta_Q * q_vals - self.beta_P * p_vals

        # Numerical stability: subtract max before exp
        prefs = prefs - np.max(prefs)
        exp_prefs = np.exp(prefs)
        probs = exp_prefs / np.sum(exp_prefs)

        # Sample according to the softmax probabilities
        return np.random.choice(self.nA, p=probs)

    # -------------------------------------------------------------------------
    # Public API used by monitor.py / interact()
    # -------------------------------------------------------------------------
    def select_action(self, state):
        """Select an action using epsilon-greedy with softmax exploration."""
        if np.random.random() < self.epsilon:
            # EXPLORE: not uniform, but softmax over Q and path memory
            action = self._softmax_explore_action(state)
        else:
            # EXPLOIT: standard greedy wrt Q(s,·)
            action = self._greedy_action(state)
        return action

    def _decay_epsilon_step(self):
        """Decay epsilon once per step."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def _decay_path_episode(self):
        """Decay the entire path-memory table at the end of each episode."""
        # path is small (≤500 reachable states * 6 actions), so iterating is fine.
        for s in list(self.path.keys()):
            self.path[s] *= self.path_decay

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and exploration bookkeeping.

        Q(s,a) ← Q(s,a) + α [ r + γ max_a' Q(s',a') − Q(s,a) ]
        """
        # 1) Update path memory: we used (state, action) once more
        self.path[state][action] += 1.0

        # 2) Standard tabular Q-learning
        q_sa = self.Q[state][action]

        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])

        td_error = target - q_sa
        self.Q[state][action] = q_sa + self.alpha * td_error

        # 3) Decay epsilon *per step* (your best-performing schedule)
        self._decay_epsilon_step()

        # 4) Episode-level bookkeeping
        if done:
            # decay novelty counts so recent actions matter more than ancient ones
            self._decay_path_episode()


# === interact() unchanged ====================================================

def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ==============================================
env = gym.make("Taxi-v3")  # <-- correct environment ID
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 8.74 || ε=0.00100


Final best average reward: 8.74


## bump alpha from 0.08 to 0.12

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Q-Learning with Step-Based Epsilon Decay (α = 0.12) ===

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3 with tuned hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        # Zero initialization (proved most stable in your experiments)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.12    # ⬆️ slightly increased from 0.08
        self.gamma = 0.99    # high discount factor

        # Epsilon-greedy exploration (step-based decay)
        self.epsilon = 1.0          # initial exploration
        self.epsilon_min = 0.001    # floor
        self.epsilon_decay = 0.99995  # per-STEP decay (your best schedule)

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy."""
        if np.random.random() < self.epsilon:
            # Explore: uniform random over actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error and update
        td_error = target - q_sa
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Decay epsilon every step (fine-grained exploration schedule)
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)


Episode 20000/20000 || Best average reward 8.76 || ε=0.00100


Final best average reward: 8.76


# A few experiments (variations of above)

## Verison A: Optimistic Init + Two-Phase Alpha (Most Promising!)

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Q-learning with optimistic init and adaptive learning rate.

        Key improvements:
        - Optimistic initialization (Q=1.5) for exploration
        - Two-phase alpha: fast learning then refinement
        - Step-based epsilon decay (proven to work)
        """
        self.nA = nA

        # CHANGE 1: Optimistic initialization
        self.Q = defaultdict(lambda: np.ones(self.nA) * 1.5)

        # CHANGE 2: Two-phase learning rate
        self.alpha_high = 0.15   # Early learning
        self.alpha_low = 0.05    # Late refinement
        self.alpha_transition = 10000  # Switch point
        self.step_count = 0

        self.gamma = 0.99

        # Keep your proven epsilon schedule
        self.epsilon = 1.0
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.99995

    @property
    def alpha(self):
        """Adaptive learning rate based on step count."""
        if self.step_count < self.alpha_transition:
            return self.alpha_high
        else:
            # Smooth transition
            progress = (self.step_count - self.alpha_transition) / 200000
            return max(self.alpha_low, self.alpha_high - progress * (self.alpha_high - self.alpha_low))

    def _greedy_action(self, state):
        """Return greedy action with random tie-breaking."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Epsilon-greedy action selection."""
        if np.random.random() < self.epsilon:
            return np.random.choice(self.nA)
        else:
            return self._greedy_action(state)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update with adaptive alpha."""
        self.step_count += 1

        q_sa = self.Q[state][action]
        target = reward if done else reward + self.gamma * np.max(self.Q[next_state])
        td_error = target - q_sa

        # Use adaptive alpha
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Decay epsilon per step
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best avg reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f} α={agent.alpha:.3f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 TARGET REACHED in {i_episode} episodes!")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== Run Variant A ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"\nFinal best average reward: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg reward 8.84 || ε=0.00100 α=0.050


Final best average reward: 8.84


## Version B: Lower Gamma (More Myopic)


In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Q-learning with lower gamma for myopic planning.

        Key change: gamma=0.95 makes agent prioritize immediate rewards more.
        This can help find shorter paths in grid worlds.
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.ones(self.nA) * 1.0)  # Mild optimism

        self.alpha = 0.1
        self.gamma = 0.95  # CHANGE: Lower discount factor

        self.epsilon = 1.0
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.99995

    def _greedy_action(self, state):
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.choice(self.nA)
        else:
            return self._greedy_action(state)

    def step(self, state, action, reward, next_state, done):
        q_sa = self.Q[state][action]
        target = reward if done else reward + self.gamma * np.max(self.Q[next_state])
        td_error = target - q_sa
        self.Q[state][action] = q_sa + self.alpha * td_error
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100):
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state
            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best avg reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 TARGET REACHED in {i_episode} episodes!")
            break

    print("\n")
    return avg_rewards, best_avg_reward


env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"\nFinal best average reward: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg reward 8.67 || ε=0.00100


Final best average reward: 8.67


## Variant C: Three-Phase Epsilon (Exploration → Exploitation → Refinement)

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Q-learning with three-phase epsilon decay.

        Phase 1 (0-300k steps): Fast exploration (ε: 1.0 → 0.1)
        Phase 2 (300k-600k steps): Moderate exploitation (ε: 0.1 → 0.01)
        Phase 3 (600k+ steps): Refinement (ε: 0.01 → 0.001)
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.ones(self.nA) * 1.2)

        self.alpha = 0.1
        self.gamma = 0.99

        self.epsilon = 1.0
        self.epsilon_min = 0.001
        self.step_count = 0

        # Three-phase thresholds
        self.phase1_steps = 300000
        self.phase2_steps = 600000

    def _get_epsilon_decay(self):
        """Adaptive epsilon decay based on training phase."""
        if self.step_count < self.phase1_steps:
            return 0.999992  # Fast decay: 1.0 → ~0.1 in 300k steps
        elif self.step_count < self.phase2_steps:
            return 0.999995  # Medium decay: 0.1 → ~0.01 in next 300k
        else:
            return 0.999998  # Slow decay: 0.01 → 0.001

    def _greedy_action(self, state):
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.choice(self.nA)
        else:
            return self._greedy_action(state)

    def step(self, state, action, reward, next_state, done):
        self.step_count += 1

        q_sa = self.Q[state][action]
        target = reward if done else reward + self.gamma * np.max(self.Q[next_state])
        td_error = target - q_sa
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Adaptive epsilon decay
        decay_rate = self._get_epsilon_decay()
        self.epsilon = max(self.epsilon_min, self.epsilon * decay_rate)


def interact(env, agent, num_episodes=20000, window=100):
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state
            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best avg reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 TARGET REACHED in {i_episode} episodes!")
            break

    print("\n")
    return avg_rewards, best_avg_reward


env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"\nFinal best average reward: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg reward 6.68 || ε=0.05054


Final best average reward: 6.68


# Optimistic Q-Learning with Decaying Alpha

This strategy employs two main mechanisms to overcome local optima and ensure convergence:

1. Optimistic Initialization: The Q-table is initialized with a high value (e.g., 5.0). Since most rewards are less than 5, the agent is incentivized to explore every state-action pair aggressively to 'correct' the overestimation, ensuring comprehensive coverage of the state space.

2. Decaying Learning Rate ($\alpha$): The learning rate starts high (e.g., 0.5) for aggressive initial learning and then decays slowly per episode down to a minimum (e.g., 0.01). This allows for quick, large updates early on and guarantees finer, stable convergence later in the training process.

The $\epsilon$-decay remains step-based, as this proved effective in the previous iteration.

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimistic Q-Learning with Decaying Alpha (Dual Decay) ===

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent using Optimistic Initialization and Dual Decay.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: Initialize optimistically to 5.0.
        # This encourages initial exploration, as the agent seeks to disprove
        # the overly high reward estimates.
        self.Q = defaultdict(lambda: np.full(self.nA, 5.0))

        # Q-learning hyperparameters
        self.alpha = 0.5      # Learning rate: Start high for fast initial updates
        self.alpha_min = 0.01 # Minimum learning rate
        self.alpha_decay = 0.9995 # Decay factor per EPISODE
        self.gamma = 0.99     # Discount factor (kept high)

        # Epsilon-greedy exploration hyperparameters (Per-Step Decay)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        self.epsilon_decay = 0.99995  # Decay per STEP (slower overall)

        self.episode_count = 0    # Counts completed episodes

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy."""
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def _update_alpha(self):
        """Decay alpha once per episode."""
        self.episode_count += 1
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update with dual decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target calculation
        if done:
            target = reward
        else:
            # Max over next state's action-values (Q-learning)
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update using current alpha
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Update exploration rate (per step)
        self._update_epsilon()

        # Update learning rate (per episode)
        if done:
            self._update_alpha()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # Agent performs Q-update and decay
            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # Monitor progress (now includes current alpha)
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best avg reward {best_avg_reward:.2f} || "
            f"α={agent.alpha:.4f} | ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        # check if task is solved
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best avg reward 8.67 || α=0.0100 | ε=0.00100


Final best average reward: 8.67


## Conservative Stable Q-Learning (Slower $\epsilon$ Decay)

We are leveraging the successful configuration (step-based $\epsilon$-decay, non-optimistic initialization) and making two minor, stabilizing adjustments: slightly reducing the learning rate ($\alpha$) to ensure convergence is smooth, and slightly slowing the $\epsilon$-decay to prolong the final, necessary exploration of hard-to-reach states.Change 1 (Stability): Reduce $\alpha$ from $0.08$ to $\mathbf{0.06}$ for more conservative updates.Change 2 (Exploration): Slow $\epsilon$-decay from $0.99995$ to $\mathbf{0.99998}$ to keep the agent searching for the optimal path for a longer fraction of the 20,000 episodes.

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === SARSA (State-Action-Reward-State-Action) Control ===

class Agent:
    def __init__(self, nA=6):
        """Tabular SARSA agent for Taxi-v3.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # SARSA hyperparameters (reverted to successful Q-learning baseline)
        self.alpha = 0.08     # Learning rate
        self.gamma = 0.99     # Discount factor

        # Epsilon-greedy exploration hyperparameters (Per-Step Decay)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        self.epsilon_decay = 0.99995 # Slower decay per STEP

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy."""
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, next_action, done):
        """SARSA update and step-based epsilon decay.

        SARSA update uses the Q-value of the *next action (next_action)*.
        Q(s,a) ← Q(s,a) + α [ r + γ Q(s',a') - Q(s,a) ]
        """
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target calculation (SARSA: uses Q[next_state][next_action])
        if done:
            target = reward
        else:
            # CRITICAL CHANGE: Use Q-value of the actual action taken in next_state
            target = reward + self.gamma * self.Q[next_state][next_action]

        # TD error
        td_error = target - q_sa

        # SARSA update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Update exploration rate (per step)
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance, adapted for SARSA's sequential nature."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        # 1. Select the initial action A using the current policy (e-greedy)
        action = agent.select_action(state)

        while True:
            # 2. Take action A, observe R, S'
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # 3. Select next action A' using the current policy (e-greedy)
            next_action = agent.select_action(next_state)

            # 4. Agent performs internal update (uses S, A, R, S', A', done)
            agent.step(state, action, reward, next_state, next_action, done)

            samp_reward += reward

            # 5. Move to the next time step (S=S', A=A')
            state = next_state
            action = next_action

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # Monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best avg reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        # check if task is solved
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best avg reward 8.68 || ε=0.00100


Final best average reward: 8.68


In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.06     # Learning rate: Slightly reduced from 0.1
        self.gamma = 0.99     # Discount factor (kept high)

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration: 0.99995 is common
        self.epsilon_decay = 0.99998

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}", # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 8.64 || ε=0.00100


Final best average reward: 8.64


## Dual Step Decay Q-Learning

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Dual Step-Decay Q-Learning ===

class Agent:
    def __init__(self, nA=6):
        """Tabular Q-learning agent with Dual Step-Decay for Alpha and Epsilon.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA). Zero init for stability.
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = 0.2      # Learning rate: Start high for fast initial updates
        self.alpha_min = 0.01 # Minimum learning rate for fine tuning
        # Extremely slow decay per step. Ensures alpha approaches 0.01 smoothly.
        self.alpha_decay = 0.999999
        self.gamma = 0.99     # Discount factor

        # Epsilon-greedy exploration hyperparameters (Per-Step Decay)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        self.epsilon_decay = 0.99995 # Slower decay per STEP (proven effective)

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy."""
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def _update_alpha(self):
        """Decay alpha once per step, very slowly."""
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update with dual step-based decay for alpha and epsilon.

        Q(s,a) ← Q(s,a) + α [ r + γ max_a' Q(s',a') - Q(s,a) ]
        """
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target calculation (Q-Learning: uses max Q[next_state])
        if done:
            target = reward
        else:
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update using current (and decaying) alpha
        self.Q[state][action] = q_sa + self.alpha * td_error

        # Update exploration rate (per step)
        self._update_epsilon()

        # Update learning rate (per step)
        self._update_alpha()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance, adapted back for Q-Learning."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # Agent selects action
            action = agent.select_action(state)
            # Take action, observe R, S'
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # Agent performs update and dual decay (uses S, A, R, S', done)
            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # Monitor progress (now includes current alpha and epsilon)
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best avg reward {best_avg_reward:.2f} || "
            f"α={agent.alpha:.6f} | ε={agent.epsilon:.5f}",
            end=""
        )
        sys.stdout.flush()

        # check if task is solved
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best avg reward 8.80 || α=0.146774 | ε=0.00100


Final best average reward: 8.8


# Optimized Q-learning (claude)

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Optimized Q-learning agent for Taxi-v3 leaderboard.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # CRITICAL: Optimistic initialization encourages exploration
        # Initialize Q-values to a positive number rather than 0
        self.Q = defaultdict(lambda: np.ones(self.nA) * 2.0)

        # --- Hyperparameters (optimized for 9.27+ target) ---
        # Learning rate: start higher, decay faster
        self.alpha = 0.15         # initial learning rate (higher than your 0.1)
        self.alpha_min = 0.005    # lower minimum (more exploitation late)
        self.alpha_decay = 0.99995  # slower decay

        self.gamma = 0.99         # discount factor (keep this)

        # Epsilon: aggressive decay schedule
        self.epsilon = 1.0
        self.epsilon_min = 0.001   # much lower minimum (almost pure exploitation)
        self.epsilon_decay = 0.99995  # slower, steadier decay

        self.episode = 0

    def select_action(self, state):
        """ε-greedy action selection (simple and effective).

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        if np.random.random() > self.epsilon:
            # Exploit: choose best known action
            return np.argmax(self.Q[state])
        else:
            # Explore: random action
            return np.random.choice(self.nA)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update with per-step parameter decay.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        # Q-learning: off-policy TD update
        best_next_q = 0.0 if done else np.max(self.Q[next_state])
        td_target = reward + self.gamma * best_next_q
        td_error = td_target - self.Q[state][action]

        # Update Q-value
        self.Q[state][action] += self.alpha * td_error

        # Decay parameters every step (smoother than per-episode)
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)

        if done:
            self.episode += 1


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.4f} α={agent.alpha:.4f}",
            end=""
        )
        sys.stdout.flush()

        # Target threshold
        if best_avg_reward >= 9.27:
            print(f"\n🎯 TARGET REACHED in {i_episode} episodes!", end="")
            break

        if i_episode == num_episodes:
            print("\n")

    return avg_rewards, best_avg_reward


# ==== main entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)

avg_rewards, best_avg_reward = interact(env, agent)
print("\n\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 4.70 || ε=0.0010 α=0.0050



Final best average reward: 4.7


Sarsa:

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Optimized SARSA agent for Taxi-v3 leaderboard.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # CRITICAL: Optimistic initialization encourages exploration
        # Initialize Q-values to a positive number rather than 0
        self.Q = defaultdict(lambda: np.ones(self.nA) * 2.0)

        # --- Hyperparameters (optimized for 9.27+ target) ---
        # Learning rate: start higher, decay faster
        self.alpha = 0.15         # initial learning rate
        self.alpha_min = 0.005    # lower minimum (more exploitation late)
        self.alpha_decay = 0.99995  # slower decay

        self.gamma = 0.99         # discount factor

        # Epsilon: aggressive decay schedule
        self.epsilon = 1.0
        self.epsilon_min = 0.001   # much lower minimum (almost pure exploitation)
        self.epsilon_decay = 0.99995  # slower, steadier decay

        self.episode = 0

    def select_action(self, state):
        """ε-greedy action selection (simple and effective).

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        if np.random.random() > self.epsilon:
            # Exploit: choose best known action
            return np.argmax(self.Q[state])
        else:
            # Explore: random action
            return np.random.choice(self.nA)

    def step(self, state, action, reward, next_state, done):
        """SARSA: on-policy TD update.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        # SARSA: on-policy update using the action we WILL take
        if done:
            next_q = 0.0
        else:
            # Use the action you WILL take (on-policy)
            next_action = self.select_action(next_state)
            next_q = self.Q[next_state][next_action]

        td_target = reward + self.gamma * next_q
        td_error = td_target - self.Q[state][action]
        self.Q[state][action] += self.alpha * td_error

        # Decay parameters every step (smoother than per-episode)
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)

        if done:
            self.episode += 1


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.4f} α={agent.alpha:.4f}",
            end=""
        )
        sys.stdout.flush()

        # Target threshold
        if best_avg_reward >= 9.27:
            print(f"\n🎯 TARGET REACHED in {i_episode} episodes!", end="")
            break

        if i_episode == num_episodes:
            print("\n")

    return avg_rewards, best_avg_reward


# ==== main entry point ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)

avg_rewards, best_avg_reward = interact(env, agent)
print("\n\nFinal best average reward:", best_avg_reward)

Episode 20000/20000 || Best average reward 5.65 || ε=0.0010 α=0.0050



Final best average reward: 5.65


More experiments:

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """SARSA agent - closely matching Andy Harless's 9.26 solution.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA

        # Optimistic initialization - Andy uses 5.0
        self.Q = defaultdict(lambda: np.ones(self.nA) * 5.0)

        # Learning rate - constant (Andy doesn't decay it)
        self.alpha = 0.1

        # Discount factor
        self.gamma = 0.99

        # Epsilon schedule - slow decay per EPISODE
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9999  # very slow

        self.episode = 0

    def select_action(self, state):
        """ε-greedy action selection."""
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[state])
        else:
            return np.random.choice(self.nA)

    def step(self, state, action, reward, next_state, done):
        """SARSA update (on-policy)."""
        # SARSA: use next action from current policy
        if done:
            next_q = 0.0
        else:
            next_action = self.select_action(next_state)
            next_q = self.Q[next_state][next_action]

        # TD update
        td_target = reward + self.gamma * next_q
        td_error = td_target - self.Q[state][action]
        self.Q[state][action] += self.alpha * td_error

        # Decay epsilon once per episode
        if done:
            self.episode += 1
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # Print every 100 episodes
        if i_episode % 100 == 0:
            print(
                f"\rEpisode {i_episode}/{num_episodes} || "
                f"Best avg reward {best_avg_reward:.2f} || "
                f"ε={agent.epsilon:.4f}",
                end=""
            )
            sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 TARGET REACHED in {i_episode} episodes!")
            break

    print("\n")
    return avg_rewards, best_avg_reward


# ==== Main ====
env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"Final best average reward: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg reward 1.75 || ε=0.1353

Final best average reward: 1.75


Another try from claude:

In [ ]:
# SIMPLE BASELINE TEST - Run this first!
import numpy as np
import gymnasium as gym
from collections import defaultdict, deque

env = gym.make("Taxi-v3")

# Extremely simple Q-learning (no tricks)
Q = defaultdict(lambda: np.zeros(6))
alpha = 0.1
gamma = 0.99
epsilon = 1.0

rewards_per_episode = []

for episode in range(10000):
    state, _ = env.reset()
    total_reward = 0

    while True:
        # Epsilon-greedy
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state])

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        total_reward += reward

        # Q-learning update
        best_next = 0 if done else np.max(Q[next_state])
        Q[state][action] += alpha * (reward + gamma * best_next - Q[state][action])

        state = next_state
        if done:
            break

    rewards_per_episode.append(total_reward)
    epsilon = max(0.01, epsilon * 0.999)

    if episode % 1000 == 0:
        avg = np.mean(rewards_per_episode[-100:]) if len(rewards_per_episode) >= 100 else np.mean(rewards_per_episode)
        print(f"Episode {episode}: Avg reward (last 100) = {avg:.2f}, ε={epsilon:.3f}")

print(f"\nFinal average (last 100): {np.mean(rewards_per_episode[-100:]):.2f}")

Episode 0: Avg reward (last 100) = -812.00, ε=0.999
Episode 1000: Avg reward (last 100) = -61.45, ε=0.367
Episode 2000: Avg reward (last 100) = -0.82, ε=0.135
Episode 3000: Avg reward (last 100) = 4.54, ε=0.050
Episode 4000: Avg reward (last 100) = 7.31, ε=0.018
Episode 5000: Avg reward (last 100) = 8.06, ε=0.010
Episode 6000: Avg reward (last 100) = 7.72, ε=0.010
Episode 7000: Avg reward (last 100) = 7.28, ε=0.010
Episode 8000: Avg reward (last 100) = 7.83, ε=0.010
Episode 9000: Avg reward (last 100) = 7.61, ε=0.010

Final average (last 100): 7.61


In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6, algorithm='q-learning', **params):
        """Flexible RL agent for testing different algorithms and hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        - algorithm: 'q-learning' or 'sarsa'
        - params: hyperparameters (q_init, alpha, gamma, epsilon_decay, epsilon_min)
        """
        self.nA = nA
        self.algorithm = algorithm

        # Hyperparameters with defaults
        q_init = params.get('q_init', 0.0)
        self.Q = defaultdict(lambda: np.ones(self.nA) * q_init)

        self.alpha = params.get('alpha', 0.1)
        self.gamma = params.get('gamma', 0.99)

        self.epsilon = 1.0
        self.epsilon_min = params.get('epsilon_min', 0.01)
        self.epsilon_decay = params.get('epsilon_decay', 0.999)

        self.episode = 0

    def select_action(self, state):
        """ε-greedy action selection."""
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[state])
        else:
            return np.random.choice(self.nA)

    def step(self, state, action, reward, next_state, done):
        """Update Q-values using specified algorithm."""

        if self.algorithm == 'q-learning':
            # Q-learning: off-policy, use max Q-value
            next_q = 0.0 if done else np.max(self.Q[next_state])
        else:  # sarsa
            # SARSA: on-policy, use actual next action
            if done:
                next_q = 0.0
            else:
                next_action = self.select_action(next_state)
                next_q = self.Q[next_state][next_action]

        # TD update (same for both)
        td_target = reward + self.gamma * next_q
        td_error = td_target - self.Q[state][action]
        self.Q[state][action] += self.alpha * td_error

        # Decay epsilon
        if done:
            self.episode += 1
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100, verbose=True):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # Print progress
        if verbose and i_episode % 100 == 0:
            print(
                f"\rEpisode {i_episode}/{num_episodes} || "
                f"Best avg reward {best_avg_reward:.2f} || "
                f"ε={agent.epsilon:.4f}",
                end=""
            )
            sys.stdout.flush()

        if best_avg_reward >= 9.27:
            if verbose:
                print(f"\n🎯 TARGET REACHED in {i_episode} episodes!")
            break

    if verbose:
        print()
    return best_avg_reward


def grid_search():
    """Test different hyperparameter combinations."""

    configs = [
        # Classic Q-learning variants
        {'name': 'Q-learning (zero init)', 'algorithm': 'q-learning', 'q_init': 0.0,
         'alpha': 0.1, 'epsilon_decay': 0.999, 'epsilon_min': 0.01},

        {'name': 'Q-learning (optimistic)', 'algorithm': 'q-learning', 'q_init': 2.0,
         'alpha': 0.1, 'epsilon_decay': 0.999, 'epsilon_min': 0.01},

        {'name': 'Q-learning (fast decay)', 'algorithm': 'q-learning', 'q_init': 0.0,
         'alpha': 0.1, 'epsilon_decay': 0.995, 'epsilon_min': 0.001},

        # SARSA variants
        {'name': 'SARSA (zero init)', 'algorithm': 'sarsa', 'q_init': 0.0,
         'alpha': 0.1, 'epsilon_decay': 0.999, 'epsilon_min': 0.01},

        {'name': 'SARSA (optimistic)', 'algorithm': 'sarsa', 'q_init': 2.0,
         'alpha': 0.1, 'epsilon_decay': 0.999, 'epsilon_min': 0.01},

        {'name': 'SARSA (Andy-style)', 'algorithm': 'sarsa', 'q_init': 5.0,
         'alpha': 0.1, 'epsilon_decay': 0.9999, 'epsilon_min': 0.01},

        # High learning rate variants
        {'name': 'Q-learning (high alpha)', 'algorithm': 'q-learning', 'q_init': 0.0,
         'alpha': 0.2, 'epsilon_decay': 0.999, 'epsilon_min': 0.01},

        {'name': 'SARSA (high alpha)', 'algorithm': 'sarsa', 'q_init': 0.0,
         'alpha': 0.2, 'epsilon_decay': 0.999, 'epsilon_min': 0.01},
    ]

    print("=" * 80)
    print("GRID SEARCH - Testing Multiple Configurations")
    print("=" * 80)

    results = []

    for config in configs:
        name = config.pop('name')
        print(f"\n\nTesting: {name}")
        print("-" * 80)

        env = gym.make("Taxi-v3")
        agent = Agent(nA=env.action_space.n, **config)
        best_reward = interact(env, agent, num_episodes=20000, window=100, verbose=True)

        results.append({
            'name': name,
            'config': config,
            'best_reward': best_reward
        })

        print(f"Final: {best_reward:.2f}")

    # Summary
    print("\n\n" + "=" * 80)
    print("SUMMARY - All Results")
    print("=" * 80)
    results.sort(key=lambda x: x['best_reward'], reverse=True)

    for i, result in enumerate(results, 1):
        print(f"{i}. {result['name']}: {result['best_reward']:.2f}")
        print(f"   Config: {result['config']}")

    print("\n🏆 WINNER:", results[0]['name'], f"with {results[0]['best_reward']:.2f}")

    return results


# Run grid search
results = grid_search()

GRID SEARCH - Testing Multiple Configurations


Testing: Q-learning (zero init)
--------------------------------------------------------------------------------
Episode 20000/20000 || Best avg reward 8.52 || ε=0.0100
Final: 8.52


Testing: Q-learning (optimistic)
--------------------------------------------------------------------------------
Episode 20000/20000 || Best avg reward 8.69 || ε=0.0100
Final: 8.69


Testing: Q-learning (fast decay)
--------------------------------------------------------------------------------
Episode 20000/20000 || Best avg reward 8.78 || ε=0.0010
Final: 8.78


Testing: SARSA (zero init)
--------------------------------------------------------------------------------
Episode 20000/20000 || Best avg reward 8.45 || ε=0.0100
Final: 8.45


Testing: SARSA (optimistic)
--------------------------------------------------------------------------------
Episode 20000/20000 || Best avg reward 8.23 || ε=0.0100
Final: 8.23


Testing: SARSA (Andy-style)
----------------

Expected sarsa

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym  # updated: use gymnasium instead of gym


class Agent:

    def __init__(self, nA=6):
        """Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Hyperparameters for Expected SARSA
        self.alpha = 0.1      # learning rate
        self.gamma = 0.99     # discount factor

        # Epsilon-greedy parameters
        self.epsilon = 1.0        # initial exploration rate
        self.epsilon_min = 0.01   # minimum exploration rate
        self.epsilon_decay = 0.9999  # decay rate per step

        self.episode_count = 0

    def select_action(self, state):
        """Given the state, select an action using epsilon-greedy policy.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        # Epsilon-greedy action selection
        if np.random.random() < self.epsilon:
            # Explore: choose random action
            return np.random.choice(self.nA)
        else:
            # Exploit: choose best action based on current Q-values
            return np.argmax(self.Q[state])

    def step(self, state, action, reward, next_state, done):
        """Update the agent's knowledge using Expected SARSA update rule.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        # Expected SARSA target:
        # target = r + γ * E_{a' ~ π(·|s')} [ Q(s', a') ]
        # where π is the current ε-greedy policy.

        if done:
            # Terminal state: no future rewards
            target = reward
        else:
            q_next = self.Q[next_state]

            # current greedy action in next_state
            best_action = np.argmax(q_next)

            # ε-greedy policy over next_state
            policy_probs = np.ones(self.nA) * (self.epsilon / self.nA)
            policy_probs[best_action] += (1.0 - self.epsilon)

            # Expected value under current policy
            expected_q_next = np.dot(policy_probs, q_next)

            target = reward + self.gamma * expected_q_next

        # Update Q-value towards the target
        self.Q[state][action] += self.alpha * (target - self.Q[state][action])

        # Decay epsilon after each step (same as before so you can compare)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance.

    Params
    ======
    - env: instance of Gymnasium's Taxi-v3 environment
    - agent: instance of class Agent
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || Best average reward {best_avg_reward:.2f}",
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward


# ==== main notebook entry point ====
# create environment and agent, then run training
env = gym.make("Taxi-v3")  # updated: Taxi-v3 with Gymnasium
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print("\nFinal best average reward:", best_avg_reward)


Episode 20000/20000 || Best average reward 8.71


Final best average reward: 8.71


## Pure Sarsa:

- Q_init = 5.0 (very optimistic)
- alpha = 0.1 (constant, NO decay)
- gamma = 0.99
- epsilon: 1.0 → 0.01, decay = 0.9999 per episode

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Pure SARSA - exact clone of Andy Harless's winning solution."""
        self.nA = nA

        # Very optimistic initialization
        self.Q = defaultdict(lambda: np.ones(self.nA) * 5.0)

        # Constant learning rate (no decay!)
        self.alpha = 0.1

        # High discount factor
        self.gamma = 0.99

        # Epsilon decay per EPISODE (not step!)
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9999  # Very slow, per episode

    def select_action(self, state):
        """Simple epsilon-greedy."""
        if np.random.random() < self.epsilon:
            return np.random.choice(self.nA)
        else:
            return np.argmax(self.Q[state])

    def step(self, state, action, reward, next_state, done):
        """SARSA update (on-policy)."""

        # SARSA: use the action you will actually take next
        if done:
            next_q = 0.0
        else:
            next_action = self.select_action(next_state)
            next_q = self.Q[next_state][next_action]

        # TD update
        target = reward + self.gamma * next_q
        self.Q[state][action] += self.alpha * (target - self.Q[state][action])

        # Decay epsilon ONCE per episode
        if done:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100):
    """Standard interaction loop."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        if i_episode % 100 == 0:
            print(
                f"\rEpisode {i_episode}/{num_episodes} || "
                f"Best avg {best_avg_reward:.2f} || "
                f"ε={agent.epsilon:.4f}",
                end=""
            )
            sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 REACHED in {i_episode} episodes!")
            break

    print()
    return avg_rewards, best_avg_reward


env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"Final: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg 1.66 || ε=0.1353
Final: 1.66


## Q-Learning with Episode-Based Decay

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Q-learning with EPISODE-based epsilon decay."""
        self.nA = nA

        # Moderate optimistic initialization
        self.Q = defaultdict(lambda: np.ones(self.nA) * 2.0)

        self.alpha = 0.1
        self.gamma = 0.99

        # Episode-based epsilon decay
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995  # Faster than Andy's, per episode

    def select_action(self, state):
        """Epsilon-greedy with tie-breaking."""
        if np.random.random() < self.epsilon:
            return np.random.choice(self.nA)
        else:
            q_vals = self.Q[state]
            return np.random.choice(np.flatnonzero(q_vals == np.max(q_vals)))

    def step(self, state, action, reward, next_state, done):
        """Q-learning update."""

        # Q-learning: off-policy, use max
        next_q = 0.0 if done else np.max(self.Q[next_state])
        target = reward + self.gamma * next_q
        self.Q[state][action] += self.alpha * (target - self.Q[state][action])

        # Decay epsilon ONCE per episode
        if done:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100):
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state
            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        if i_episode % 100 == 0:
            print(f"\rEpisode {i_episode}/{num_episodes} || Best avg {best_avg_reward:.2f} || ε={agent.epsilon:.4f}", end="")
            sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 REACHED in {i_episode} episodes!")
            break

    print()
    return avg_rewards, best_avg_reward


env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"Final: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg 8.41 || ε=0.0100
Final: 8.41


## Hybrid - Your Best Base + Episode Decay

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym


class Agent:
    def __init__(self, nA=6):
        """Take your working 8.96 solution but switch to EPISODE-based decay."""
        self.nA = nA

        # Your working settings
        self.Q = defaultdict(lambda: np.zeros(self.nA))  # Conservative init that worked
        self.alpha = 0.08  # Your value that worked
        self.gamma = 0.99

        # CRITICAL CHANGE: Episode-based epsilon decay
        self.epsilon = 1.0
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.9993  # Tuned for episode-based (faster than 0.9999)

    def _greedy_action(self, state):
        """Your tie-breaking implementation."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        best_actions = np.flatnonzero(q_vals == max_q)
        return np.random.choice(best_actions)

    def select_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.choice(self.nA)
        else:
            return self._greedy_action(state)

    def step(self, state, action, reward, next_state, done):
        """Q-learning - same as your working version."""
        q_sa = self.Q[state][action]
        target = reward if done else reward + self.gamma * np.max(self.Q[next_state])
        self.Q[state][action] = q_sa + self.alpha * (target - q_sa)

        # CHANGE: Decay epsilon per EPISODE, not per step
        if done:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def interact(env, agent, num_episodes=20000, window=100):
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state
            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        print(f"\rEpisode {i_episode}/{num_episodes} || Best avg {best_avg_reward:.2f} || ε={agent.epsilon:.5f}", end="")
        sys.stdout.flush()

        if best_avg_reward >= 9.27:
            print(f"\n🎯 REACHED in {i_episode} episodes!")
            break

    print()
    return avg_rewards, best_avg_reward


env = gym.make("Taxi-v3")
agent = Agent(nA=env.action_space.n)
avg_rewards, best_avg_reward = interact(env, agent)
print(f"Final: {best_avg_reward:.2f}")

Episode 20000/20000 || Best avg 8.65 || ε=0.00100
Final: 8.65


## Manus Version 1

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6, alpha=0.08, gamma=0.99, epsilon_decay=0.99995):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        - alpha: learning rate
        - gamma: discount factor
        - epsilon_decay: step-based decay rate for epsilon
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = alpha     # Learning rate
        self.gamma = gamma     # Discount factor

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration
        self.epsilon_decay = epsilon_decay

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}", # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward

def run_experiment(alpha, gamma, epsilon_decay, num_episodes=20000):
    """Runs a single experiment with specified hyperparameters."""
    env = gym.make("Taxi-v3")
    agent = Agent(nA=env.action_space.n, alpha=alpha, gamma=gamma, epsilon_decay=epsilon_decay)
    print(f"\n--- Running Experiment: α={alpha}, γ={gamma}, ε_decay={epsilon_decay} ---")
    _, best_avg_reward = interact(env, agent, num_episodes=num_episodes)
    print(f"Final best average reward: {best_avg_reward:.4f}")
    return best_avg_reward

if __name__ == '__main__':
    # Optimized Configuration:
    # - Alpha (Learning Rate): 0.06 (Slightly lower than 0.08 for better convergence)
    # - Gamma (Discount Factor): 0.995 (Slightly higher than 0.99 to value future rewards more)
    # - Epsilon Decay: 0.99998 (Slower decay to ensure exploration lasts longer)
    # - Episodes: 30000 (Increased to allow the slower decay to converge)

    best_avg_reward = run_experiment(alpha=0.06, gamma=0.995, epsilon_decay=0.99998, num_episodes=30000)
    print(f"\nFinal best average reward from optimized configuration: {best_avg_reward:.4f}")


--- Running Experiment: α=0.06, γ=0.995, ε_decay=0.99998 ---
Episode 30000/30000 || Best average reward 8.80 || ε=0.00100

Final best average reward: 8.8000

Final best average reward from optimized configuration: 8.8000


## Version 2

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6, alpha=0.08, gamma=0.99, epsilon_decay=0.99995):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        - alpha: learning rate
        - gamma: discount factor
        - epsilon_decay: step-based decay rate for epsilon
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = alpha     # Learning rate
        self.gamma = gamma     # Discount factor

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration
        self.epsilon_decay = epsilon_decay

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}", # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward

def run_experiment(alpha, gamma, epsilon_decay, num_episodes=20000):
    """Runs a single experiment with specified hyperparameters."""
    env = gym.make("Taxi-v3")
    agent = Agent(nA=env.action_space.n, alpha=alpha, gamma=gamma, epsilon_decay=epsilon_decay)
    print(f"\n--- Running Experiment: α={alpha}, γ={gamma}, ε_decay={epsilon_decay} ---")
    _, best_avg_reward = interact(env, agent, num_episodes=num_episodes)
    print(f"Final best average reward: {best_avg_reward:.4f}")
    return best_avg_reward

if __name__ == '__main__':
    # Replicating the user's best run to establish a baseline.
    best_avg_reward = run_experiment(alpha=0.08, gamma=0.99, epsilon_decay=0.99995, num_episodes=20000)
    print(f"\nFinal best average reward from baseline configuration: {best_avg_reward:.4f}")


--- Running Experiment: α=0.08, γ=0.99, ε_decay=0.99995 ---
Episode 20000/20000 || Best average reward 9.01 || ε=0.00100

Final best average reward: 9.0100

Final best average reward from baseline configuration: 9.0100


## Version 3

In [ ]:
import numpy as np
import math
import sys
from collections import defaultdict, deque
import gymnasium as gym

# === Optimized Epsilon-Decay Q-Learning ===
# Strategy: Slower, step-based epsilon decay and slightly tuned hyperparameters
# to ensure more thorough exploration of the state space.

class Agent:
    def __init__(self, nA=6, alpha=0.08, gamma=0.99, epsilon_decay=0.99995):
        """Tabular Q-learning agent for Taxi-v3 with optimized hyperparameters.

        Params
        ======
        - nA: number of actions available to the agent
        - alpha: learning rate
        - gamma: discount factor
        - epsilon_decay: step-based decay rate for epsilon
        """
        self.nA = nA

        # Q-table: maps state -> array of action-values (size nA)
        self.Q = defaultdict(lambda: np.zeros(self.nA))

        # Q-learning hyperparameters
        self.alpha = alpha     # Learning rate
        self.gamma = gamma     # Discount factor

        # Epsilon-greedy exploration hyperparameters (CRITICAL CHANGES)
        self.epsilon = 1.0        # Initial exploration rate
        self.epsilon_min = 0.001  # Minimum exploration rate
        # Slower, step-based decay for prolonged exploration
        self.epsilon_decay = epsilon_decay

    def _greedy_action(self, state):
        """Return a greedy action for a given state, breaking ties randomly."""
        q_vals = self.Q[state]
        max_q = np.max(q_vals)
        # All actions with the max Q-value
        best_actions = np.flatnonzero(q_vals == max_q)
        # Break ties randomly among equally good actions
        return np.random.choice(best_actions)

    def select_action(self, state):
        """Select an action using epsilon-greedy policy.

        This method now also handles the step-based epsilon decay.
        """
        if np.random.random() < self.epsilon:
            # Explore: uniform over all actions
            return np.random.choice(self.nA)
        else:
            # Exploit: greedy action (with random tie-breaking)
            return self._greedy_action(state)

    def _update_epsilon(self):
        """Decay epsilon once per step (not per episode)."""
        # Step-based epsilon decay schedule
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def step(self, state, action, reward, next_state, done):
        """Q-learning update and step-based epsilon decay."""
        # Current Q(s,a)
        q_sa = self.Q[state][action]

        # Target for Q-learning
        if done:
            target = reward
        else:
            # Max over next state's action-values
            target = reward + self.gamma * np.max(self.Q[next_state])

        # TD error
        td_error = target - q_sa

        # Q-learning update
        self.Q[state][action] = q_sa + self.alpha * td_error

        # CRITICAL CHANGE: Decay epsilon *per step*
        self._update_epsilon()


def interact(env, agent, num_episodes=20000, window=100):
    """Monitor agent's performance (No change needed here, it's robust)."""
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)

    # for each episode
    for i_episode in range(1, num_episodes + 1):
        # Gymnasium: reset returns (obs, info)
        state, info = env.reset()
        samp_reward = 0

        while True:
            # agent selects an action
            action = agent.select_action(state)
            # Gymnasium: step returns (obs, reward, terminated, truncated, info)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # agent performs internal updates based on sampled experience
            # Epsilon decay is now *inside* agent.step
            agent.step(state, action, reward, next_state, done)

            # update the sampled reward
            samp_reward += reward
            # update the state
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            # get average reward from last window episodes
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        # monitor progress
        print(
            f"\rEpisode {i_episode}/{num_episodes} || "
            f"Best average reward {best_avg_reward:.2f} || "
            f"ε={agent.epsilon:.5f}", # Showing more digits for epsilon decay
            end=""
        )
        sys.stdout.flush()

        # check if task is solved (Gym definition used in many examples)
        # We will use a higher threshold for the final check, but keep the original for comparison
        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.", end="")
            break

    if i_episode == num_episodes:
        print("\n")

    return avg_rewards, best_avg_reward

def run_experiment(alpha, gamma, epsilon_decay, num_episodes=20000):
    """Runs a single experiment with specified hyperparameters."""
    # Use the original Taxi-v3 environment
    env = gym.make("Taxi-v3")
    agent = Agent(nA=env.action_space.n, alpha=alpha, gamma=gamma, epsilon_decay=epsilon_decay)
    print(f"\n--- Running Experiment: α={alpha}, γ={gamma}, ε_decay={epsilon_decay} ---")
    _, best_avg_reward = interact(env, agent, num_episodes=num_episodes)
    print(f"Final best average reward: {best_avg_reward:.4f}")
    return best_avg_reward

if __name__ == '__main__':
    # Based on user insights:
    # - Fixed alpha=0.08 is good. Let's try a slightly lower one for better convergence.
    # - Gamma=0.99 is high, which is good for long-term rewards.
    # - Step-based decay is critical. Let's try a slower decay rate.

    # Configuration 1: Baseline (similar to user's best result)
    # alpha=0.08, gamma=0.99, epsilon_decay=0.99995
    # run_experiment(alpha=0.08, gamma=0.99, epsilon_decay=0.99995)

    # Configuration 2: Slower decay for more exploration
    # The agent reaches epsilon_min=0.001 after about 138,000 steps with 0.99995 decay.
    # A typical episode is around 13 steps. 20000 episodes * 13 steps/episode = 260,000 steps.
    # Let's try a decay that ensures exploration lasts longer than 20,000 episodes.
    # 0.99998: (0.001/1.0) = 0.99998^x -> x = log(0.001)/log(0.99998) ~ 345,000 steps
    # This should keep epsilon active for the full 20,000 episodes.

    # Configuration 2: Slower decay, same alpha
    # best_avg_reward_2 = run_experiment(alpha=0.08, gamma=0.99, epsilon_decay=0.99998)

    # Configuration 3: Slower decay, slightly lower alpha for better convergence
    # best_avg_reward_3 = run_experiment(alpha=0.06, gamma=0.99, epsilon_decay=0.99998)

    # Configuration 4: Slower decay, slightly higher gamma (more future-focused)
    # best_avg_reward_4 = run_experiment(alpha=0.08, gamma=0.995, epsilon_decay=0.99998)

    # We will start with the most promising configuration based on the user's insights and our analysis:
    # Slower decay (0.99998) and the proven alpha (0.08).
    best_avg_reward_final = run_experiment(alpha=0.08, gamma=0.99, epsilon_decay=0.99998, num_episodes=30000)

    # If the first run is not enough, we can try the other configurations in subsequent steps.
    # Increasing episodes to 30,000 to give the slower decay more time to converge.
    print(f"Final best average reward from best configuration: {best_avg_reward_final:.4f}")


--- Running Experiment: α=0.08, γ=0.99, ε_decay=0.99998 ---
Episode 30000/30000 || Best average reward 8.73 || ε=0.00100

Final best average reward: 8.7300
Final best average reward from best configuration: 8.7300


Andy

In [ ]:
import numpy as np
from collections import defaultdict, deque
import sys, math, random
import gymnasium as gym  # <-- IMPORTANT: Gymnasium


# ============================
#        AGENT.PY
# ============================

class Agent:

    def __init__(self, nA=6, alpha=.75, gamma=1, beta=.8, c1=0, c2=0,
                 get_epsilon=lambda i: .8*i**.999,
                 get_alpha=None, get_gamma=None, get_beta=None,
                 get_c1=None, get_c2=None):

        self.alpha_init = alpha
        self.gamma_init = gamma
        self.beta_init = beta
        self.c1_init = c1
        self.c2_init = c2

        self.get_epsilon = get_epsilon
        self.get_alpha = (lambda i:self.alpha_init) if get_alpha is None else get_alpha
        self.get_gamma = (lambda i:self.gamma_init) if get_gamma is None else get_gamma
        self.get_beta = (lambda i:self.beta_init) if get_beta is None else get_beta
        self.get_c1 = (lambda i:self.c1_init) if get_c1 is None else get_c1
        self.get_c2 = (lambda i:self.c2_init) if get_c2 is None else get_c2

        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.recent = defaultdict(lambda: np.zeros(self.nA))

        self.epsilon = self.get_epsilon(0)
        self.alpha = self.get_alpha(0)
        self.gamma = self.get_gamma(0)
        self.beta = self.get_beta(0)
        self.c1 = self.get_c1(0)
        self.c2 = self.get_c2(0)
        self.i_episode = 0


    def select_action(self, state):

        if state not in self.Q:
            return np.random.choice(self.nA)

        q = np.asarray(self.Q[state])
        r = np.asarray(self.recent[state])

        p = self.softmax(q*self.c1 - r*self.c2)

        greedy_action = q.argmax()
        random_action = np.random.choice(self.nA, p=p)

        return np.random.choice([random_action, greedy_action],
                                p=[self.epsilon, 1-self.epsilon])


    def step(self, state, action, reward, next_state, done):

        greedy_action = self.Q[next_state].argmax()

        self.Q[state][action] += self.alpha * (
            reward + self.gamma * self.Q[next_state][greedy_action] - self.Q[state][action]
        )

        self.recent[state][action] += 1

        if done:
            for st in self.recent:
                self.recent[st] = [count * self.beta for count in self.recent[st]]

            self.i_episode += 1
            self.epsilon = self.get_epsilon(self.i_episode)
            self.alpha = self.get_alpha(self.i_episode)
            self.gamma = self.get_gamma(self.i_episode)
            self.beta = self.get_beta(self.i_episode)
            self.c1 = self.get_c1(self.i_episode)
            self.c2 = self.get_c2(self.i_episode)


    @staticmethod
    def softmax(a):
        e = np.exp(a - np.max(a))
        return e / e.sum()


# ============================
#        MONITOR.PY
# ============================

def interact(env, agent, num_episodes=20000, window=100,
             show_progress=False, endline=""):

    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes+1):

        state, info = env.reset()  # Gymnasium returns (obs, info)
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)

            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)

            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= 100:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        if show_progress and not i_episode % show_progress:
            print(
                f"\rEpisode {i_episode}/{num_episodes} || epsilon={agent.epsilon:.7f}, Best avg reward {best_avg_reward}",
                end=endline
            )
            sys.stdout.flush()

        if best_avg_reward >= 9.7:
            print(f"\nEnvironment solved in {i_episode} episodes.")
            break

    return avg_rewards, best_avg_reward


# ============================
#         MAIN.PY
# ============================

n_episodes = 150000
nruns = 1
medsub = nruns // 2

beta = .7
c1 = .02
c2 = 3
alpha = .7
gamma = .5
a = -.005
b = 5e-5
eps_min = 0
epfunc = lambda i: max(eps_min, math.exp(a - b*i))

best_avg_rewards = []
local_seed = 80
start = 3

print("\n\nBegin Taxi-v3 learning...")

for i in range(start, start + nruns):

    env = gym.make("Taxi-v3")

    # Proper Gymnasium seeding
    env.reset(seed=10000*i + local_seed)
    random.seed(i + local_seed)
    np.random.seed(100*i + local_seed)

    agent = Agent(alpha=alpha, gamma=gamma, get_epsilon=epfunc,
                  c1=c1, c2=c2, beta=beta)

    avg_rewards, best_avg_reward = interact(
        env, agent, n_episodes, show_progress=10000, endline="\n"
    )

    best_avg_rewards.append(best_avg_reward)
    print(f"Run {i}/{nruns}, avg so far = {sum(best_avg_rewards)/len(best_avg_rewards)}")


print("\nLocal seed:", local_seed)
print("Average:", sum(best_avg_rewards)/len(best_avg_rewards))
print("Median:", sorted(best_avg_rewards)[medsub])
print(np.array(sorted(best_avg_rewards)))




Begin Taxi-v3 learning...
Episode 10000/150000 || epsilon=0.6035056, Best avg reward -81.52
Episode 20000/150000 || epsilon=0.3660446, Best avg reward -20.1
Episode 30000/150000 || epsilon=0.2220173, Best avg reward -4.68
Episode 40000/150000 || epsilon=0.1346603, Best avg reward 2.71
Episode 50000/150000 || epsilon=0.0816756, Best avg reward 5.14
Episode 60000/150000 || epsilon=0.0495388, Best avg reward 6.59
Episode 70000/150000 || epsilon=0.0300468, Best avg reward 7.67
Episode 80000/150000 || epsilon=0.0182243, Best avg reward 8.05
Episode 90000/150000 || epsilon=0.0110536, Best avg reward 8.34
Episode 100000/150000 || epsilon=0.0067043, Best avg reward 8.7
Episode 110000/150000 || epsilon=0.0040664, Best avg reward 8.7
Episode 120000/150000 || epsilon=0.0024664, Best avg reward 8.7
Episode 130000/150000 || epsilon=0.0014959, Best avg reward 8.71
Episode 140000/150000 || epsilon=0.0009073, Best avg reward 9.0
Episode 150000/150000 || epsilon=0.0005503, Best avg reward 9.0
Run 3/1

In [ ]:
!pip install bayesian-optimization

In [ ]:
import os
import sys
import math
from collections import defaultdict, deque

import numpy as np
import gymnasium as gym

from bayes_opt import BayesianOptimization


# ============================
#         AGENT
# ============================
class Agent:
    def __init__(self, nA=6, eps=0.1, eps_decay=0.9, alpha=0.3, gamma=0.9, mode='expected'):
        """Initialize agent with configurable hyperparameters."""
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.num_episodes = 0

        self.eps = eps
        self.min_eps = 0.0
        self.eps_decay = eps_decay

        self.alpha = alpha
        self.gamma = gamma

        self.mode = mode  # 'q-learning' or 'expected'

    def _get_probs(self, state):
        """Get action probabilities for epsilon-greedy policy."""
        max_i = np.argmax(self.Q[state])
        return [
            (1 - self.eps + self.eps / self.nA) if i == max_i else self.eps / self.nA
            for i in range(self.nA)
        ]

    def select_action(self, state):
        """Select action using epsilon-greedy policy."""
        return np.random.choice(range(self.nA), p=self._get_probs(state))

    def _update_params(self):
        """Decay epsilon after each episode."""
        self.eps = max(self.min_eps, self.eps * self.eps_decay)

    def step(self, s, a, r, s_next, done):
        """Update Q-values using Q-learning or Expected SARSA."""
        if self.mode == 'q-learning':
            a_best = np.argmax(self.Q[s_next])
            exp_val = self.Q[s_next][a_best]
        elif self.mode == 'expected':
            # Expected SARSA: weighted average over all actions
            exp_val = sum(x * y for x, y in zip(self.Q[s_next], self._get_probs(s_next)))
        else:
            raise RuntimeError('Unsupported mode')

        # TD update
        self.Q[s][a] = (1 - self.alpha) * self.Q[s][a] + self.alpha * (r + self.gamma * exp_val)

        if done:
            self.num_episodes += 1
            self._update_params()


# ============================
#       MONITOR
# ============================
def interact(env, agent, num_episodes=20000, window=100, verbose=False):
    """Monitor agent's performance."""
    avg_rewards = deque(maxlen=num_episodes)
    best_avg_reward = -math.inf
    samp_rewards = deque(maxlen=window)

    for i_episode in range(1, num_episodes + 1):
        state, info = env.reset()
        samp_reward = 0

        while True:
            action = agent.select_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            agent.step(state, action, reward, next_state, done)
            samp_reward += reward
            state = next_state

            if done:
                samp_rewards.append(samp_reward)
                break

        if i_episode >= window:
            avg_reward = np.mean(samp_rewards)
            avg_rewards.append(avg_reward)
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward

        if verbose and i_episode % 1000 == 0:
            print(f"\rEpisode {i_episode}/{num_episodes} || Best avg {best_avg_reward:.2f}", end="")
            sys.stdout.flush()

        if best_avg_reward >= 9.7:
            if verbose:
                print(f'\nEnvironment solved in {i_episode} episodes.')
            break

    if verbose:
        print()
    return avg_rewards, best_avg_reward


# ============================
#       BAYESIAN OPTIMIZATION
# ============================
def interact_wrapper(eps, eps_decay, alpha, gamma):
    """Wrapper function for Bayesian optimization."""
    env = gym.make('Taxi-v3')
    agent = Agent(
        nA=env.action_space.n,
        eps=eps,
        eps_decay=eps_decay,
        alpha=alpha,
        gamma=gamma,
        mode='expected'  # Use Expected SARSA
    )
    _, best_avg_reward = interact(env, agent, num_episodes=20000, window=100, verbose=False)
    print(f"Tested: eps={eps:.3f}, eps_decay={eps_decay:.3f}, alpha={alpha:.3f}, gamma={gamma:.3f} -> {best_avg_reward:.2f}")
    return best_avg_reward


# ============================
#        MAIN
# ============================
if __name__ == '__main__':
    # Define parameter bounds for Bayesian optimization
    pbounds = {
        'eps': (0.5, 1.0),           # Initial epsilon
        'eps_decay': (0.4, 0.95),    # Epsilon decay rate
        'alpha': (0.05, 0.3),        # Learning rate
        'gamma': (0.8, 0.99)         # Discount factor
    }

    # Initialize Bayesian optimizer
    optimizer = BayesianOptimization(
        f=interact_wrapper,
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    # Add some good starting points based on research
    optimizer.probe(
        params={'eps': 0.85, 'eps_decay': 0.6, 'alpha': 0.15, 'gamma': 0.9},
        lazy=True,
    )
    optimizer.probe(
        params={'eps': 1.0, 'eps_decay': 0.45, 'alpha': 0.05, 'gamma': 0.85},
        lazy=True,
    )
    optimizer.probe(
        params={'eps': 0.9, 'eps_decay': 0.5, 'alpha': 0.1, 'gamma': 0.9},
        lazy=True,
    )

    print("Starting Bayesian Optimization...")
    print("=" * 60)

    # Run optimization
    optimizer.maximize(
        init_points=5,    # Random exploration points
        n_iter=20         # Optimization iterations
    )

    # Print results
    print("\n" + "=" * 60)
    print("OPTIMIZATION COMPLETE")
    print("=" * 60)
    print(f"\nBest parameters found:")
    print(f"  eps:       {optimizer.max['params']['eps']:.4f}")
    print(f"  eps_decay: {optimizer.max['params']['eps_decay']:.4f}")
    print(f"  alpha:     {optimizer.max['params']['alpha']:.4f}")
    print(f"  gamma:     {optimizer.max['params']['gamma']:.4f}")
    print(f"\nBest reward: {optimizer.max['target']:.2f}")

    # Test the best configuration multiple times
    print("\n" + "=" * 60)
    print("TESTING BEST CONFIGURATION (5 runs)")
    print("=" * 60)

    best_params = optimizer.max['params']
    test_rewards = []

    for run in range(5):
        env = gym.make('Taxi-v3')
        agent = Agent(
            nA=env.action_space.n,
            eps=best_params['eps'],
            eps_decay=best_params['eps_decay'],
            alpha=best_params['alpha'],
            gamma=best_params['gamma'],
            mode='expected'
        )
        _, reward = interact(env, agent, num_episodes=20000, window=100, verbose=False)
        test_rewards.append(reward)
        print(f"Run {run+1}: {reward:.2f}")

    print(f"\nAverage over 5 runs: {np.mean(test_rewards):.2f} ± {np.std(test_rewards):.2f}")

Starting Bayesian Optimization...
|   iter    |  target   |    eps    | eps_decay |   alpha   |   gamma   |
-------------------------------------------------------------------------
Tested: eps=0.850, eps_decay=0.600, alpha=0.150, gamma=0.900 -> 8.91
| 1         | 8.91      | 0.85      | 0.6       | 0.15      | 0.9       |
Tested: eps=1.000, eps_decay=0.450, alpha=0.050, gamma=0.850 -> 8.74
| 2         | 8.74      | 1.0       | 0.45      | 0.05      | 0.85      |
Tested: eps=0.900, eps_decay=0.500, alpha=0.100, gamma=0.900 -> 8.77
| 3         | 8.77      | 0.9       | 0.5       | 0.1       | 0.9       |
Tested: eps=0.687, eps_decay=0.923, alpha=0.233, gamma=0.914 -> 8.86
| 4         | 8.86      | 0.6872700 | 0.9228928 | 0.2329984 | 0.9137451 |
Tested: eps=0.578, eps_decay=0.486, alpha=0.065, gamma=0.965 -> 8.67
| 5         | 8.67      | 0.5780093 | 0.4857969 | 0.0645209 | 0.9645734 |
Tested: eps=0.801, eps_decay=0.789, alpha=0.055, gamma=0.984 -> 8.88
| 6         | 8.88      | 0.800557